# Identifying the topics based on questions asked by Amazon customers

### What is topic modelling?
It is an unsupervised approach used for finding and observing the bunch of words (called Topics) in large clusters of texts.

### Algorithm used:
There are many algorithms available for Topic Modelling such as LSA (Latent Semantic Analysis), NMF (Non-Matrix Factorization) and LDA (Latent Dirichlet Algorithm). I'm using LDA as it provides more accurate results and scales well for large text corpuses.

### A brief intro to LDA algorithm:
LDA is a matrix factorization technique which assumes sentences are made up of mixture of words and tries to figure out what words would create those sentences in the first place. It is based on probabilistic graphical modelling. The algorithm works as follows:<br>
1. In the initialization stage, each word is assigned to a random topic.
2. Iteratively, the algorithm goes through each word and reassigns the word to a topic taking into consideration:<br>
    What’s the probability of the word belonging to a topic?<br>
    What’s the probability of the document to be generated by a topic?
[Source]('https://nlpforhackers.io/topic-modeling/')

### Data Source:
http://jmcauley.ucsd.edu/data/amazon/qa/. Click on the Automotive category file to download the json file having the customer queries.

#### Step 1: Import the required libraries

In [12]:
#Data Manipulation
import pandas as pd
import numpy as np

#Read the zip files
import gzip

#LDA model
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)
import nltk
nltk.download('wordnet')

#Visualizing LDA
import pyLDAvis
pyLDAvis.enable_notebook()

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rupamanupbanerjee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Step 2: Import the data into dataframe

In [13]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

data = getDF('qa_Automotive.json.gz')

#### Step 3: Extract the text column and assign it to an index

In [14]:
def prepare_data(file_name):
    data_text = data[['question']]
    data_text['index'] = data_text.index
    return data_text
documents=prepare_data(data)

Overview of the file

In [15]:
documents.head()

,question,index
0,What is the most useful length to get?,0
1,Are these cables made of copper or aluminum?,1
2,I bought the Red Extra Heavy Duty. Is that too...,2
3,"Hi, Being 20ft 4gauge how heavy is this?",3
4,Do these cables come with a bag?,4


#### Step 4: Reducing the words to its common  base form
Now, we need to reduce the words which are in its plural or derivational forms into its common base form. This will help us collect the similar words from all the corpuses. Now the question is how do we do that? Here, stemming and lemmatization comes to the rescue.<br><br>
**What is Stemming?** <br>
The process of chopping off the ends of words, often includes the removal of derivational fixes. E.g. cats -> cat<br>
There are two types of stemmer: SnowballStemmer and PorterStemmer. We will be using SnowballStemmer as it gives a more meaningful result compared to PorterStemmer().<br>

**What is Lemmatization?** <br>
The process of removal of inflectional endings only and returning the base or dictionary form of a word, which is known as lemma. E.g. running -> run.<br>

In [16]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
stemmer = SnowballStemmer('english')

Next, we need to remove the stopwords and words whose length is less than or equal to three as they won't add much value to our model. We will implement it using gensim library as it already has a tokenization function and built-in library of stopwords.

In [17]:
def remove_stopwords_and_shortwords(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

Let's look at an example of how it works:

In [18]:
sample = documents[documents['index'] == 25].values[0][0]

print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n Tokenized and lemmatized document: ')
print(remove_stopwords_and_shortwords(sample))

original document: 
['It', 'does', 'not', 'state', 'what', 'RPM', 'this', 'motor', 'is,', 'does', 'anyone', 'know?']


 Tokenized and lemmatized document: 
['state', 'motor', 'know']


Now, we will apply the same on our dataset.

In [19]:
processed_docs = documents['question'].map(remove_stopwords_and_shortwords)

Overview of the processed docs

In [20]:
processed_docs[:10]

0                      [use, length]
1           [cabl, copper, aluminum]
2    [buy, extra, heavi, duti, size]
3                      [gaug, heavi]
4                       [cabl, come]
5        [wire, pair, surpris, wire]
6                       [amp, handl]
7              [cabl, boost, school]
8                      [use, length]
9           [cabl, copper, aluminum]
Name: question, dtype: object

#### Step 5: Build a dictionary of words present in the dataset

In [21]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('There are total '+ str(len(dictionary))+' words in the dictionary.')

There are total 14469 words in the dictionary.


An overview of the dictionary:

In [22]:
count = 0
for index, word in dictionary.iteritems():
    print(index, word)
    count += 1
    if count > 10:
        break

0 length
1 use
2 aluminum
3 cabl
4 copper
5 buy
6 duti
7 extra
8 heavi
9 size
10 gaug


#### Step 6: Filter the dictionary based on frequency of words. 
Here, I have ignored the words which have appeared in less than 4 queries and the ones who are present in at most 50% of the documents.

In [23]:
dictionary.filter_extremes(no_below=4, no_above=0.5)
print('No. of words in the dictionary after filtering: '+ str(len(dictionary)))

No. of words in the dictionary after filtering: 4705


#### Step 7: Create a list having the index value and the frequency of the word for every question in the dataset.

In [24]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[:5]

[[(0, 1), (1, 1)],
 [(2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(8, 1), (10, 1)],
 [(3, 1), (11, 1)]]

A sample:

In [28]:
bow_doc_sample = bow_corpus[0]

for i in range(len(bow_doc_sample)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_sample[i][0], 
                                                     dictionary[bow_doc_sample[i][0]], 
                                                     bow_doc_sample[i][1]))

Word 0 ("length") appears 1 time.
Word 1 ("use") appears 1 time.


#### Step 8: Implement the LDA model

In [26]:
lda_model = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=5, id2word=dictionary, passes=5,minimum_probability=0.01, random_state=20)

In [27]:
for idx, topic in lda_model.print_topics(-1,num_words=7):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.038*"light" + 0.036*"bulb" + 0.034*"work" + 0.032*"toyota" + 0.019*"dodg" + 0.018*"long" + 0.018*"inch"
Topic: 1 
Words: 0.036*"instal" + 0.030*"come" + 0.025*"mount" + 0.024*"need" + 0.024*"wire" + 0.022*"work" + 0.021*"light"
Topic: 2 
Words: 0.032*"size" + 0.032*"tire" + 0.026*"need" + 0.019*"color" + 0.018*"order" + 0.017*"black" + 0.015*"like"
Topic: 3 
Words: 0.084*"work" + 0.036*"seat" + 0.033*"model" + 0.026*"honda" + 0.021*"ford" + 0.021*"chevi" + 0.019*"cover"
Topic: 4 
Words: 0.029*"batteri" + 0.026*"say" + 0.019*"unit" + 0.017*"plug" + 0.014*"light" + 0.014*"right" + 0.014*"charg"


#### Step 9: Visualize the topics
I have used pyLDAvis library which is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.[Source]('https://github.com/bmabey/pyLDAvis')

In [31]:
import pyLDAvis.gensim_models

In [32]:
vis1=pyLDAvis.gensim_models.prepare(lda_model,bow_corpus,dictionary, R=7)
vis1

/Users/rupamanupbanerjee/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/rupamanupbanerjee/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/rupamanupbanerjee/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/rupamanupbanerjee/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/rupamanupbanerjee/opt

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.105394 -0.157935       1        1  24.507930
4      0.241710 -0.018244       2        1  20.964482
2      0.119809  0.274177       3        1  18.593465
3     -0.322527  0.145630       4        1  18.429788
0     -0.144384 -0.243628       5        1  17.504335, topic_info=        Term          Freq         Total Category  logprob  loglift
79      work  10535.000000  10535.000000  Default   7.0000   7.0000
148   instal   3442.000000   3442.000000  Default   6.0000   6.0000
674     seat   2609.000000   2609.000000  Default   5.0000   5.0000
1464    bulb   2469.000000   2469.000000  Default   4.0000   4.0000
26     model   2389.000000   2389.000000  Default   3.0000   3.0000
...      ...           ...           ...      ...      ...      ...
385     ship   1008.079247   1008.894287   Topic5  -4.2210   1.7419
92      hole    969.968630    970.786927   Topic5  -4.2596   1.7419
50      long   1217.068908   1416.257927   Topic5  -4.0326   1.5911
84     light   2585.447549   5763.442214   Topic5  -3.2792   0.9411
79      work   2344.919650  10535.855652   Topic5  -3.3768   0.2402

[62 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
309       2  0.999516    accord
428       1  0.998638    attach
44        2  0.999596   batteri
218       3  0.999558     black
256       1  0.999277   bracket
...     ...       ...       ...
14        1  0.999751      wire
79        1  0.198275      work
79        4  0.579070      work
79        5  0.222573      work
340       3  0.998755  wrangler

[82 rows x 3 columns], R=7, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 4, 1])

#### Step 10: Intepretation of topics obtained:

Now, let's try to identify the category of questions asked from the words provided for each topic:
1. Topic 1 (install, come, mount, wire, need, light, work) - **Installation**
2. Topic 2 (battery, say, unit, plug, light, power, charge) - **Battery & Charging**
3. Topic 3 (size, tire, need, color, order, black, know) - **Tire**
4. Topic 4 (work, seat, honda, model, chevi, ford, cover) - **Model specific**
5. Topic 5 (light, bulb, work, toyota, dodge, inch, dimension) - **Cannot deduce a valid topic** <br>

There are cases when the some of the topics obtained don't make much sense. One way to fix it would be to play around the hyperparameters in order to improve the results or sometimes its okay to just ignore them. Here, we can see that words in Topic 5 are not making much sense for categorizing it into specific category.